# Population structure - PCA of the FarGen cohort

In [1]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '100g'}, tmp_dir='/home/olavur/tmp')

2021-09-28 10:28:16 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-09-28 10:28:17 WARN  Hail:37 - This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.
2021-09-28 10:28:18 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2021-09-28 10:28:18 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2021-09-28 10:28:18 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-848846b477-8tkk6:4043
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/qc/hail-20210928-1028-0.2.61-3c86d3ba497a.log


In [2]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd

## Load FarGen data

Use high-quality variants.

In [12]:
BASE_DIR = '/home/olavur/experiments/2020-11-13_fargen1_exome_analysis'
mt = hl.read_matrix_table(BASE_DIR + '/data/mt/high_quality_variants.mt')

In [13]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

Number of variants: 86300
Number of samples: 468


## Population filters

To filter rare variants, we first must calculate the minor allele frequency.

In [14]:
# The number of alleles at the site is the sum of the ploidy at each site.
# This number should be twice the number of samples.
# If there are missing genotype calls, the number of alleles will be less.
AN_exprs = hl.agg.sum(mt.GT.ploidy)
mt = mt.annotate_rows(AN=AN_exprs)

# Calculate the number of alternate alleles at each site.
AC_exprs = hl.agg.sum(mt.GT.n_alt_alleles())
mt = mt.annotate_rows(AC=AC_exprs)

# Calculate the alternate allele frequency.
mt = mt.annotate_rows(AF=mt.AC / mt.AN)

Remove variants with minor allele frequency under 0.01. Note that we remove only very rare variants, because common variants can be insufficient to describe fine-scale population structure at a subpopulation scale.

In [15]:
maf_filter = 0.01
mt = mt.filter_rows((mt.AF > maf_filter) & (mt.AF < (1 - maf_filter)))

## Filter indels

Remove all indels from the dataset.

In [16]:
mt = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))

## LD pruning

Before we calculate LD, we remove multi-allelic sites.

**NOTE:** I can do LD pruning on multi-allelic sites using [split_multi()](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.split_multi).

In [17]:
mt = mt.filter_rows(hl.len(mt.alleles) == 2)

Prune variants with $r^2 > 0.2$ within a 500 000 basepair window.

**NOTE:** this $r^2$ value is quite arbitrary. It is the default from the Hail method and I did not try other values.

In [18]:
pruned_variant_table = hl.ld_prune(mt.GT, r2=0.2, bp_window_size=500000)
mt = mt.filter_rows(hl.is_defined(pruned_variant_table[mt.row_key]))

2021-09-28 10:29:19 Hail: INFO: ld_prune: running local pruning stage with max queue size of 401850 variants
[Stage 1:===================================================>       (7 + 1) / 8]2021-09-28 10:29:21 Hail: INFO: wrote table with 21318 rows in 8 partitions to /home/olavur/tmp/YXRXzYh4j7kwaCVIMeGmTY
    Total size: 681.02 KiB
    * Rows: 681.01 KiB
    * Globals: 11.00 B
    * Smallest partition: 2347 rows (75.35 KiB)
    * Largest partition:  2907 rows (92.71 KiB)
[Stage 3:=================================================>         (5 + 1) / 6]2021-09-28 10:29:28 Hail: INFO: Wrote all 6 blocks of 21318 x 468 matrix with block size 4096.
[Stage 7:===============================================>          (9 + 2) / 11]2021-09-28 10:30:02 Hail: INFO: wrote table with 6 rows in 11 partitions to /home/olavur/tmp/maHwwR6t6Ig2QznnhGX8jE
    Total size: 294.85 KiB
    * Rows: 445.00 B
    * Globals: 294.41 KiB
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  2 rows (8

Make a checkpoint, caching all operations done on the matrix table.

In [20]:
if True:
    mt = mt.checkpoint('/home/olavur/tmp/ld_pruned.ht')
else:
    mt = hl.read_matrix_table('/home/olavur/tmp/ld_pruned.ht')

[Stage 11:==================================================>       (7 + 1) / 8]2021-09-28 10:30:54 Hail: INFO: wrote matrix table with 21312 rows and 468 columns in 8 partitions to /home/olavur/tmp/ld_pruned.ht
    Total size: 155.30 MiB
    * Rows/entries: 155.25 MiB
    * Columns: 52.00 KiB
    * Globals: 11.00 B
    * Smallest partition: 2345 rows (17.00 MiB)
    * Largest partition:  2905 rows (21.10 MiB)


In [21]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

Number of variants: 21312
Number of samples: 468


This may be a lot of variants to run a PCA analysis on, but as mentioned previously, we do not want to remove too many rare variants as these can hold a lot of information about fine-scale population structure.

## Annotate birth place region

Import data containing the birthplace ID of all FarGen participants. We will annotate the matrix table with these IDs.

First we read a table containing the birthplace but with the 'RIN' participant ID. We will have to map this 'RIN' ID to an 'FN' ID.

In [22]:
rin_birthplace_ht = hl.import_table(BASE_DIR + '/data/metadata/birthplace/rin_region.csv', delimiter=',')
# Rename "ind" to "rin".
# Convert the region variable to float.
rin_birthplace_ht = rin_birthplace_ht.transmute(rin=rin_birthplace_ht.ind, birthplace=hl.float64(rin_birthplace_ht.region))

rin_birthplace_ht = rin_birthplace_ht.key_by(rin_birthplace_ht.rin)

2021-09-28 10:31:01 Hail: INFO: Reading table without type imputation
  Loading field 'ind' as type str (not specified)
  Loading field 'region' as type str (not specified)


Import table with 'RIN' IDs and corresponding 'FN' IDs.

In [23]:
fargen_rin_ht = hl.import_table(BASE_DIR + '/data/metadata/birthplace/fargen_rin_samplename.csv', delimiter=',')
fargen_rin_ht = fargen_rin_ht.key_by(fargen_rin_ht.rin)

2021-09-28 10:31:01 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']
2021-09-28 10:31:01 Hail: INFO: Reading table without type imputation
  Loading field 'rin' as type str (not specified)
  Loading field 'sample' as type str (not specified)


Make a table with 'RIN', 'FN' and birthplace.

In [24]:
# Annotate the table with the birthplace by the samplenames.
samplename_birthplace_ht = rin_birthplace_ht.annotate(samplename=fargen_rin_ht[rin_birthplace_ht.rin].sample)
samplename_birthplace_ht = samplename_birthplace_ht.key_by(samplename_birthplace_ht.samplename)

Finally, we an annotate the matrix table with birthplace of samples.

In [25]:
mt = mt.annotate_cols(birthplace = samplename_birthplace_ht[mt.s].birthplace)

Count the number of samples in each region. Note that `birthplace=6` means that we do not know the birthplace of the sample.

In [26]:
cols_ht = mt.cols()
result = (cols_ht.group_by(cols_ht.birthplace)
    .aggregate(count = hl.agg.count()))
result.to_pandas()

2021-09-28 10:31:01 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2021-09-28 10:31:01 Hail: WARN: Name collision: field 'count' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['count']
2021-09-28 10:31:02 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-28 10:31:02 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 16:>                                                         (0 + 1) / 1]2021-09-28 10:31:05 Hail: INFO: Coerced sorted dataset
2021-09-28 10:31:05 Hail: INFO: Coerced dataset with out-of-order partitions.
[Stage 20:>                                                         (0 + 7) / 7]

,birthplace,count
0,1.0,69
1,2.0,99
2,3.0,177
3,4.0,28
4,5.0,17
5,6.0,34
6,NaN,44


## Filter related individuals

Estimate the relatedness between the samples by the PC-Relate method, with a minimum alternate allele frequency of 0.001.

In [27]:
pc_rel = hl.pc_relate(mt.GT, 0.001, k=2, statistics='kin')

[Stage 21:>                                                         (0 + 8) / 8]2021-09-28 10:31:08 Hail: INFO: hwe_normalized_pca: running PCA using 21312 variants.
[Stage 22:==============>                                           (2 + 6) / 8]2021-09-28 10:31:10 Hail: INFO: pca: running PCA with 2 components...
[Stage 426:======================================>                  (4 + 2) / 6]2021-09-28 10:31:30 Hail: INFO: Wrote all 6 blocks of 21312 x 468 matrix with block size 4096.


Plot all the relatedness coefficients in a histogram to get an overview.

In [28]:
p = hl.plot.histogram(pc_rel.kin, title='Histogram of kinship coefficient')
show(p)

2021-09-28 10:31:31 Hail: INFO: wrote matrix with 3 rows and 21312 columns as 6 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-WlIg9kPQtZOeabY7IiDdNF.bm
2021-09-28 10:31:31 Hail: INFO: wrote matrix with 21312 rows and 468 columns as 6 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-Vp2po5irnuBP3IVXgt1y1V.bm
[Stage 429:>                                                        (0 + 1) / 1]2021-09-28 10:31:39 Hail: INFO: wrote matrix with 468 rows and 468 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-TmQfR1xBpLdUlxgjD393cp.bm
[Stage 430:>                                                        (0 + 1) / 1]2021-09-28 10:31:46 Hail: INFO: wrote matrix with 468 rows and 468 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-U7xTBVjbdwinyoA4OMSqzY.bm
2021-09-28 10:31:46 Hail: INFO: wrote matrix with 468 rows and 468 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-tclNjmlrnXFG8MkyQrhUAx.bm
2021-09-28 

In [29]:
pairs = pc_rel.filter(pc_rel['kin'] > 2**(-4))

Then we find the maximal independent set, consistent of the samples to remove.

In [30]:
related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, keep=False)

2021-09-28 10:32:04 Hail: INFO: wrote matrix with 3 rows and 21312 columns as 6 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-tUxstYCgxkSt7UGcMQHwPa.bm
2021-09-28 10:32:04 Hail: INFO: wrote matrix with 21312 rows and 468 columns as 6 blocks of size 4096 to /home/olavur/tmp/pcrelate-write-read-Y6l6UQ6b40BkgiHLh28VN8.bm
[Stage 445:>                                                        (0 + 1) / 1]2021-09-28 10:32:12 Hail: INFO: wrote matrix with 468 rows and 468 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-oLyYDlWaREgsRaolnkd5LF.bm
[Stage 446:>                                                        (0 + 1) / 1]2021-09-28 10:32:19 Hail: INFO: wrote matrix with 468 rows and 468 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-dmLjHmBDtqbdyGU6jACvRl.bm
2021-09-28 10:32:19 Hail: INFO: wrote matrix with 468 rows and 468 columns as 1 block of size 4096 to /home/olavur/tmp/pcrelate-write-read-PAYiRQeLT7lOYRB2j9GZR6.bm
2021-09-28 

Now we filter these individuals from the matrix table.

In [31]:
mt = mt.filter_cols(hl.is_defined(related_samples_to_remove[mt.col_key]), keep=False)

Make a checkpoint, caching all operations done on the matrix table.

In [33]:
if True:
    mt = mt.checkpoint('/home/olavur/tmp/rel_pruned.ht')
else:
    mt = hl.read_matrix_table('/home/olavur/tmp/rel_pruned.ht')

2021-09-28 10:32:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-28 10:32:36 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 456:>                                                        (0 + 1) / 1]2021-09-28 10:32:38 Hail: INFO: Coerced sorted dataset
2021-09-28 10:32:38 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 461:>                                                        (0 + 8) / 8]2021-09-28 10:32:40 Hail: INFO: wrote matrix table with 21312 rows and 374 columns in 8 partitions to /home/olavur/tmp/rel_pruned.ht
    Total size: 126.05 MiB
    * Rows/entries: 126.00 MiB
    * Columns: 42.64 KiB
    * Globals: 11.00 B
    * Smallest partition: 2345 rows (13.80 MiB)
    * Largest partition:  2905 rows (17.13 MiB)


In [34]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

Number of variants: 21312
Number of samples: 374


## Compute PCA

In [35]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(mt.GT, k=4)

2021-09-28 10:32:50 Hail: INFO: hwe_normalized_pca: running PCA using 21312 variants.
2021-09-28 10:32:51 Hail: INFO: pca: running PCA with 4 components...
[Stage 477:>                                                        (0 + 2) / 2]

In [36]:
mt = mt.annotate_cols(scores = scores[mt.s].scores)

In [37]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=hl.str(mt.birthplace),
                    title='PCA', xlabel='PC1', ylabel='PC2')
p.plot_width = 800
p.plot_height = 600
show(p)

Above are the two first principal components plotted against eachother. Note that there are four outlier samples. These outliers may be due to the quality of these samples, so we shall invesigate them closer below.

As we see below, there seems to be nothing abnormal about these four samples. They may therefore potentially be *ancestry outliers*.

In [66]:
mt = mt.annotate_cols(pc_outliers = mt.scores[1] > 0.2)

In [67]:
exprs_list = [('# heterozygotes', mt.sample_qc.n_het), ('Ti/Tv rate', mt.sample_qc.r_ti_tv), ('Call rate', mt.sample_qc.call_rate), ('# singletons', mt.sample_qc.n_singleton)]
plot_list = []
for name, exprs in exprs_list:
    p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, exprs, label=mt.pc_outliers, xlabel='DP mean', ylabel=name, legend=True)
    p.plot_width = 800
    p.plot_height = 500
    plot_list.append(p)

In [68]:
show(gridplot(plot_list, ncols=2, plot_width=600, plot_height=400))

## Remove potential ancestry outliers from full dataset

In further analysis of the dataset, we do not want to include these four potential ancestry outliers (PAO). We will therefore load the full dataset, discard the PAOs, and write the data again.

In [69]:
# Potential ancestry outliers.
pao_list = mt.filter_cols(mt.pc_outliers).s.collect()

print('Number of potential ancestry outliers: {n}'.format(n=len(pao_list)))

Number of potential ancestry outliers: 8


In [70]:
# Read original matrix table.
full_mt = hl.read_matrix_table(BASE_DIR + '/data/mt/high_quality_variants.mt')

# Annotate with the list of potential ancestry outliers.
full_mt = full_mt.annotate_globals(pao_list = pao_list)

# Remove the potential ancestry outliers.
pao_removed_mt = full_mt.filter_cols(~full_mt.pao_list.contains(full_mt.s))

In [71]:
n_variants, n_samples = pao_removed_mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

Number of variants: 86300
Number of samples: 460


Update variant QC information, as this has changed when we have removed some samples. Also remove invariant sites.

In [72]:
pao_removed_mt = hl.variant_qc(pao_removed_mt)

pao_removed_mt = pao_removed_mt.filter_rows(pao_removed_mt.variant_qc.AC[1] > 0)

In [73]:
n_variants, n_samples = pao_removed_mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

[Stage 628:>                                                        (0 + 8) / 8]

Number of variants: 86254
Number of samples: 460


Write the resulting matrix table to file.

In [56]:
if False:
    pao_removed_mt.write(BASE_DIR + '/data/mt/high_quality_variants_pao_removed.mt', overwrite=True)

[Stage 611:==========================================>              (6 + 2) / 8]2021-09-28 10:37:06 Hail: INFO: wrote matrix table with 86254 rows and 460 columns in 8 partitions to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/high_quality_variants_pao_removed.mt
    Total size: 574.79 MiB
    * Rows/entries: 574.75 MiB
    * Columns: 48.99 KiB
    * Globals: 59.00 B
    * Smallest partition: 10019 rows (67.33 MiB)
    * Largest partition:  11871 rows (77.58 MiB)


## PCA without outliers

These PCA outerliers prevent us from detecting population structure in the PCA above. Whether they are ancestry outliers or whether something else is at play, we must remove them to be able to detect population structure.

In [78]:
mt = mt.filter_cols(~mt.pc_outliers)

Now we just do the PCA again, and see if PC 1 and 2 show signs of population structure.

In [79]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(mt.GT, k=4)

2021-09-28 10:43:38 Hail: INFO: hwe_normalized_pca: running PCA using 21312 variants.
2021-09-28 10:43:39 Hail: INFO: pca: running PCA with 4 components...
[Stage 765:>                                                        (0 + 8) / 8]

In [80]:
mt = mt.annotate_cols(scores = scores[mt.s].scores)

In [81]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=hl.str(mt.birthplace),
                    title='PCA', xlabel='PC1', ylabel='PC2')
p.plot_width = 800
p.plot_height = 600
show(p)

Region number | Region name(s)
-----|-----
1 | Norðoyggjar
2 | Eysturoy og Norðstreymoy
3 | Suðurstreymoy
4 | Vágar og Mykines
5 | Sandoy, Skúvoy, Stóra Dímun
6 | Suðuroy

From this PCA, it looks like there is very little population structure. But I'm not so convinced, I see this as a failure to detect population structure, not a success to show that there is no population structure.

## Summary

In this notebook we have:

* Investigate population structure in the cohort
* Removed potential ancestry outliers from the data